## Example of online analysis using OnACID

Complete pipeline (motion correction + source extraction + deconvolution) for running OnACID.
Two-photon mesoscope data kindly provided by the Tolias lab, Baylor college of medicine.

In [ ]:
try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

from IPython.display import display, clear_output
import glob
import logging
import numpy as np
import os
import scipy
import cv2

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import caiman as cm
from caiman.source_extraction import cnmf as cnmf
from caiman.paths import caiman_datadir
from caiman.utils.utils import download_demo

import bokeh.plotting as bpl
try:
       from bokeh.io import vform, hplot
except:
       # newer version of bokeh does not use vform & hplot, instead uses column & row
       from bokeh.layouts import column as vform
       from bokeh.layouts import row as hplot
from bokeh.models import CustomJS, ColumnDataSource, Slider

bpl.output_notebook()

## First download the data

The function ```download_demo``` will look for the datasets ```Tolias_mesoscope_*.hdf5``` inside the folder specified by the variable ```fld_name``` and will download the files if they do not exist. Note that you must be in the main CaImAn folder to run this demo

In [ ]:
fld_name = 'Mesoscope'                              # folder inside ./example_movies where files will be saved
download_demo('Tolias_mesoscope_1.hdf5',fld_name)
download_demo('Tolias_mesoscope_2.hdf5',fld_name)
download_demo('Tolias_mesoscope_3.hdf5',fld_name)

folder_name = os.path.join(caiman_datadir(), 'example_movies', fld_name) # folder where files are located
extension = 'hdf5'                                  # extension of files
fnames = glob.glob(folder_name + '/*' + extension)       # read all files to be processed 

print(fnames)                                          # your list of files should look something like this

## Set up some parameters

Here we set up some parameters for running OnACID. We use the same `params` object as in batch processing with CNMF.

In [ ]:
fr = 15                                                             # frame rate (Hz)
decay_time = 0.5                                                    # approximate length of transient event in seconds
gSig = (4,4)                                                        # expected half size of neurons
p = 1                                                               # order of AR indicator dynamics
min_SNR = 2.5                                                       # minimum SNR for accepting new components
rval_thr = 0.85                                                     # correlation threshold for new component inclusion
ds_factor = 1                                                       # spatial downsampling factor (increases speed but may lose some fine structure)
gnb = 2                                                             # number of background components
gSig = tuple(np.ceil(np.array(gSig)/ds_factor).astype('int'))       # recompute gSig if downsampling is involved
mot_corr = True                                                     # flag for online motion correction 
pw_rigid = False                                                    # flag for pw-rigid motion correction (slower but potentially more accurate)
max_shifts_online = np.ceil(10./ds_factor).astype('int')            # maximum allowed shift during motion correction
sniper_mode = False                                                 # flag using a CNN to detect new neurons (o/w space correlation is used)
init_batch = 200                                                    # number of frames for initialization (presumably from the first file)
expected_comps = 500                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
K = 2                                                               # initial number of components
epochs = 2                                                          # number of passes over the data
show_movie = True                                                   # show the movie with the results as the data gets processed

params_dict = {'fnames': fnames,
               'fr': fr,
               'decay_time': decay_time,
               'gSig': gSig,
               'p': p,
               'min_SNR': min_SNR,
               'rval_thr': rval_thr,
               'ds_factor': ds_factor,
               'nb': gnb,
               'motion_correct': mot_corr,
               'init_batch': init_batch,
               'init_method': 'bare',
               'normalize': True,
               'expected_comps': expected_comps,
               'sniper_mode': sniper_mode,
               'K': K,
               'epochs': epochs,
               'max_shifts_online': max_shifts_online,
               'pw_rigid': pw_rigid,
               'show_movie': show_movie}
opts = cnmf.params.CNMFParams(params_dict=params_dict)

## Now run the CaImAn online algorithm (OnACID).

The first ```initbatch``` frames are used for initialization purposes. The initialization method chosen here `bare` will only search for a small number of neurons and is mostly used to initialize the background components. Initialization with the full CNMF can also be used by choosing `cnmf`.

We first create an `OnACID` object located in the module `online_cnmf` and we pass the parameters similarly to the case of batch processing. We then run the algorithm using the `fit_online` method.

In [ ]:
cnm = cnmf.online_cnmf.OnACID(params=opts)
cnm.fit_online()

## Optionally save results and do some plotting

In [ ]:
logging.info('Number of components: ' + str(cnm.estimates.A.shape[-1]))
Cn = cm.load(fnames[0], subindices=slice(0,500)).local_correlations(swap_dim=False)
cnm.estimates.plot_contours(img=Cn)

## View components

Now inspect the components extracted by OnACID. Note that if single pass was used then several components would be non-zero only for the part of the time interval indicating that they were detected online by OnACID.

Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
cnm.estimates.nb_view_components(img=Cn, denoised_color='red');

## Save data
TO DO